In [4]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids, and separate the dataset:

In [5]:
from implementations import *

In [6]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)


In [7]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [8]:
logistic = True
tX_list, ids_list, y_list = separate_dataset(tX, ids, y, logistic)
tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test) 

In [9]:
tX_list, tX_test_list, y_list = clean_data(tX_list, tX_test_list, y_list)

## GRID SEARCH RIDGE REGRESSION

In [10]:
function = reg_logistic_regression

degrees = range(1, 5)
lambdas = np.logspace(-6, 0, 5)
gammas = np.logspace(-3, 0, 5)

degree_vec = []
lambda_vec = []
gamma_vec = []

for i in range(6):
    print('Dataset part {l}:'.format(l = i))
    rmse_te, BestDeg, BestLambda, BestGamma = grid_search(y_list[i], tX_list[i], function, 4, degrees, lambdas, gammas, dataset = i)
    degree_vec.append(BestDeg)
    lambda_vec.append(BestLambda)
    gamma_vec.append(BestGamma)
    print('     Best degree: {d}'.format(d = BestDeg))
    print('     Best lambda: {m}'.format(m = BestLambda))
    print('     Best gamma: {m}'.format(m = BestGamma))
    print('     Loss: {lo}'.format(lo = rmse_te))

print(degree_vec)
print(lambda_vec)
print(gamma_vec)

Dataset part 0:
     Best degree: 2
     Best lambda: 1e-06
     Best gamma: 0.1778279410038923
     Loss: 6166.25
Dataset part 1:
     Best degree: 3
     Best lambda: 1e-06
     Best gamma: 0.1778279410038923
     Loss: 14748.5
Dataset part 2:
     Best degree: 2
     Best lambda: 1e-06
     Best gamma: 0.1778279410038923
     Loss: 1719.75
Dataset part 3:
     Best degree: 3
     Best lambda: 1e-06
     Best gamma: 0.1778279410038923
     Loss: 13402.25
Dataset part 4:
     Best degree: 2
     Best lambda: 1e-06
     Best gamma: 0.1778279410038923
     Loss: 999.75
Dataset part 5:
     Best degree: 3
     Best lambda: 3.1622776601683795e-05
     Best gamma: 0.1778279410038923
     Loss: 13613.5
[2, 3, 2, 3, 2, 3]
[1e-06, 1e-06, 1e-06, 1e-06, 1e-06, 3.1622776601683795e-05]
[0.1778279410038923, 0.1778279410038923, 0.1778279410038923, 0.1778279410038923, 0.1778279410038923, 0.1778279410038923]


In [11]:
weights_list = []
loss_list = []
mat_tX_test_list = []
max_iter= 3000

for i in range(6):
    mat_tX, mat_tX_test = build_poly_log(tX_list[i], degree_vec[i], tX_test_list[i], i)
    initial_w = np.zeros((mat_tX.shape[1], 1))
    #w, l = ridge_regression(y_list[i], mat_tX, lambda_vec[i])
    w, l = reg_logistic_regression(y_list[i], mat_tX, initial_w, max_iter, lambda_vec[i], gamma_vec[i])
    weights_list.append(w)
    loss_list.append(l)
    mat_tX_test_list.append(mat_tX_test)    
    print(l)
 

0.15579393394156515
0.4267048025313744
0.2307474533335349
0.5208113502970543
0.2364715084205718
0.4556061574518068


In [12]:
# Eval

y_pred_list = separated_eval(weights_list, mat_tX_test_list, logistic) 

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3], y_pred_list[4], y_pred_list[5]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3], ids_test_list[4], ids_test_list[5]))

OUTPUT_PATH = 'result.csv'
create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)